In [75]:
from tensorflow import keras
from keras.datasets import fashion_mnist
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras import optimizers
from keras import losses
from keras import metrics
from keras.applications.vgg16 import VGG16
from keras import models
from keras.applications.vgg16 import preprocess_input
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization
from keras.preprocessing.image import img_to_array, array_to_img
from keras.utils import to_categorical


In [76]:
data = keras.datasets.fashion_mnist
(train_img, train_labels), (test_img, test_labels) = data.load_data()
train_img = np.array(train_img)
train_labels = np.array(train_labels)
test_img = np.array(test_img)
test_labels = np.array(test_labels)
print(train_img.shape, train_labels.shape, test_img.shape, test_labels.shape)

train_img = train_img.reshape((60000, 28, 28))
train_img = train_img.astype("float32") / 1000.
test_img = test_img.reshape((10000, 28, 28))
test_img = test_img.astype("float32") / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print(train_img.shape, train_labels.shape, test_img.shape, test_labels.shape)
print('------------------')
print(train_img.shape)
# transform to rgb as required by VGG
train_img = tf.image.grayscale_to_rgb(tf.expand_dims(train_img, axis=3))
test_img = tf.image.grayscale_to_rgb(tf.expand_dims(test_img, axis=3))


#print(len(x_train), len(y_train))

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
(60000, 28, 28) (60000, 10) (10000, 28, 28) (10000, 10)
------------------
(60000, 28, 28)


In [77]:
train_img = tf.image.resize_with_pad(train_img, 32, 32)
test_img = tf.image.resize_with_pad(test_img, 32, 32)
print(train_img) #  60000, 28, 28, 3
print('------------------')
print(test_img) # 10000, 28, 28, 3

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
 

In [78]:
train_img = tf.expand_dims(train_img, axis=0)
test_img = tf.expand_dims(test_img, axis=0)
print(train_img.shape)
print('------------------')
print(test_img.shape)

(1, 60000, 32, 32, 3)
------------------
(1, 10000, 32, 32, 3)


In [79]:
#preprocessing as required by VGG16
train_img = preprocess_input(train_img)
test_img = preprocess_input(test_img)
print(train_img.shape)
print('------------------')
print(test_img.shape)

(1, 60000, 32, 32, 3)
------------------
(1, 10000, 32, 32, 3)


In [80]:
vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))


In [81]:
layer_dict = dict([(layer.name, layer) for layer in vgg16.layers])
output = layer_dict['block3_pool'].output

x = keras.layers.Flatten()(vgg16.output)
x = keras.layers.Dense(10, activation='relu')(x)
#x = keras.layers.Dense(10)(x)

final = keras.models.Model(inputs=vgg16.input, outputs=x)
for layer in final.layers[:10]:
  layer.trainable = False

print(test_img.shape)


(1, 10000, 32, 32, 3)


In [82]:
print(train_img.shape)   #shape=(1, 1, 1, 1, 60000, 32, 32, 3), dtype=float32)
print('-------train_img-----------')
print(test_img.shape)# shape=(1, 1, 1, 1, 60000, 32, 32, 3), dtype=float32)
print('--------test_img----------')
print(train_labels.shape)
print('--------train_labels----------')
print(test_labels.shape)
print('--------test_labels----------')
#b = train_img.T
#print(b.shape)
print('--------b.shape----------')
final.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
final.fit(train_img, test_img, epochs=5, batch_size=64)
#final.fit(np.array(train_img), np.array(train_labels), verbose=0, epochs=100)
#train_img, train_labels), (test_img, test_labels)



(1, 60000, 32, 32, 3)
-------train_img-----------
(1, 10000, 32, 32, 3)
--------test_img----------
(60000, 10)
--------train_labels----------
(10000, 10)
--------test_labels----------
--------b.shape----------
Epoch 1/5


ValueError: in user code:

    File "C:\Users\Zhanna\PycharmProjects\pythonProject\venv\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Zhanna\PycharmProjects\pythonProject\venv\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Zhanna\PycharmProjects\pythonProject\venv\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Zhanna\PycharmProjects\pythonProject\venv\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Zhanna\PycharmProjects\pythonProject\venv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Zhan